In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

import warnings
warnings.filterwarnings('ignore')

In [6]:
class MF(object):
    """
    docstring for CF
    """
    def __init__(self, Y_data, K, lam = 0.1, Xinit = None, Winit = None,
                learning_rate = 0.5, max_iter = 1000, print_every = 100, user_based = 1):
        self.Y_raw_data = Y_data
        self.K = K
        # regularization parameter
        self.lam = lam
        # learning rate for gradient descent
        self.learning_rate = learning_rate
        # maximum number of iterations
        self.max_iter = max_iter
        # print results after print_every iterations
        self.print_every = print_every
        # user-based or item-based
        self.user_based = user_based
        # number of users, items, and ratings. Remember to add 1 since id starts from 0
        self.n_users = int(np.max(Y_data[:, 0])) + 1
        self.n_items = int(np.max(Y_data[:, 1])) + 1
        self.n_ratings = Y_data.shape[0]
        
        if Xinit is None: # new
            self.X = np.random.randn(self.n_items, K)
        else: # or from saved data
            self.X = Xinit
            
        if Winit is None:
            self.W = np.random.randn(K, self.n_users)
        else: # from saved data
            self.W = Winit
            
        # normalized data, update later in normalized_Y function
        self.Y_data_n = self.Y_raw_data.copy()
        
    def normalize_Y(self):
        if self.user_based:
            user_col = 0
            item_col = 1
            n_objects = self.n_users
            
        # if we want to normalize based on item, just switch first two columns of data
        else: # item bas
            user_col = 1
            item_col = 0
            n_objects = self.n_items
            
        users = self.Y_raw_data[:, user_col]
        self.mu = np.zeros((n_objects,))
        for n in range(n_objects):
            # row indices of rating done by user n
            # since indices need to be integers, we need to convert
            ids = np.where(users == n)[0].astype(np.int32)
            # indices of all ratings associated with user n
            item_ids = self.Y_data_n[ids, item_col]
            # and the corresponding ratings
            ratings = self.Y_data_n[ids, 2]
            # take mean
            m = np.mean(ratings)
            if np.isnan(m):
                m = 0 # to avoid empty array and nan value
            self.mu[n] = m
            # normalize
            self.Y_data_n[ids, 2] = ratings - self.mu[n]
    
    # Tính giá trị hàm mất mát
    def loss(self):
        L = 0
        for i in range(self.n_ratings):
            # user, item, rating
            n, m, rate = int(self.Y_data_n[i, 0]), int(self.Y_data_n[i, 1]), self.Y_data_n[i, 2]
            L += 0.5*(rate - self.X[m, :].dot(self.W[:, n]))**2
            
        # take average
        L /= self.n_ratings
        # regularization, don't ever forget this
        L += 0.5*self.lam*(np.linalg.norm(self.X, 'fro') + np.linalg.norm(self.W, 'fro'))
        return L
    
    def get_items_rated_by_user(self, user_id):
        """
        get all items which are rated by user user_id, and the corresponding ratings
        """
        ids = np.where(self.Y_data_n[:, 0] == user_id)[0]
        item_ids = self.Y_data_n[ids, 1].astype(np.int32) # indices need to be integers
        ratings = self.Y_data_n[ids, 2]
        return (item_ids, ratings)
    
    def get_users_who_rate_item(self, item_id):
        """
        get all users who rated item item_id and get the corresponding ratings
        """
        ids = np.where(self.Y_data_n[:, 1] == item_id)[0]
        user_ids = self.Y_data_n[ids, 0].astype(np.int32)
        ratings = self.Y_data_n[ids, 2]
        return (user_ids, ratings)
    
    # Cập nhật X, W:
    def updateX(self):
        for m in range(self.n_items):
            user_ids, ratings = self.get_users_who_rate_item(m)
            Wm = self.W[:, user_ids]
            # gradient
            grad_xm = -(ratings - self.X[m, :].dot(Wm)).dot(Wm.T)/self.n_ratings + self.lam*self.X[m, :]
            self.X[m, :] -= self.learning_rate*grad_xm.reshape((self.K,))
            
    def updateW(self):
        for n in range(self.n_users):
            item_ids, ratings = self.get_items_rated_by_user(n)
            Xn = self.X[item_ids, :]
            # gradient
            grad_wn = -Xn.T.dot(ratings - Xn.dot(self.W[:, n]))/self.n_ratings + self.lam*self.W[:, n]
            self.W[:, n] -= self.learning_rate*grad_wn.reshape((self.K,))
            
    # Train model
    def fit(self):
        self.normalize_Y()
        for it in range(self.max_iter):
            self.updateX()
            self.updateW()
            if (it + 1) % self.print_every == 0:
                rmse_train = self.evaluate_RMSE(self.Y_raw_data)
                print("iter = ", it + 1, ", loss = ", self.loss(), ", RMSE train = ", rmse_train)
                
    # Predict
    def pred(self, u, i):
        """
        predict the rating of user u for item i if you need the un
        """
        u = int(u)
        i = int(i)
        if self.user_based:
            bias = self.mu[u]
        else:
            bias = self.mu[i]
        pred = self.X[i, :].dot(self.W[:, u]) + bias
        
        # truncate if results are out of range [0, 5]
        if pred < 0:
            return 0
        if pred > 5:
            return 5
        return pred
    
    def pred_for_user(self, user_id):
        """
        predict ratings one user give all unrated items
        """
        ids = np.where(self.Y_data_n[:, 0] == user_id)[0]
        items_rated_by_u = self.Y_data_n[ids, 1].tolist()
        
        y_pred = self.X.dot(self.W[:, user_id]) + self.mu[user_id]
        predicted_ratings = []
        for i in range(self.n_items):
            if i not in items_rated_by_u:
                predicted_ratings.append((i, y_pred[i]))
                
        return predicted_ratings
    
    # Evaluate by Root Mean Square Error:
    def evaluate_RMSE(self, rate_test):
        n_tests = rate_test.shape[0]
        SE = 0 # squared error
        for n in range(n_tests):
            pred = self.pred(rate_test[n, 0], rate_test[n, 1])
            SE += (pred - rate_test[n, 2])**2
            
        RMSE = np.sqrt(SE/n_tests)
        return RMSE

### Áp dụng lên MovieLens 100k

In [7]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('./data/ml-100k/ub.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('./data/ml-100k/ub.test', sep='\t', names=r_cols, encoding='latin-1')

rate_train = ratings_base.as_matrix()
rate_test = ratings_test.as_matrix()

# indices start from 0
rate_train[:, :2] -= 1
rate_test[:, :2] -= 1

In [8]:
# Kết quả nếu sử dụng cách chuẩn hóa dựa trên user:
rs = MF(rate_train, K = 10, lam = .1, print_every = 10,
       learning_rate = 0.75, max_iter = 100, user_based = 1)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print("\nUser-based MF, RMSE = ", RMSE)

iter =  10 , loss =  5.657003948397523 , RMSE train =  1.2105229281312695
iter =  20 , loss =  2.6395928062641483 , RMSE train =  1.038204214929954
iter =  30 , loss =  1.3434492392077713 , RMSE train =  1.029556067071639
iter =  40 , loss =  0.7529050219316648 , RMSE train =  1.0292157498066743
iter =  50 , loss =  0.4822509285285253 , RMSE train =  1.029211157327278
iter =  60 , loss =  0.3581369241366188 , RMSE train =  1.0292134349728985
iter =  70 , loss =  0.3012193811923922 , RMSE train =  1.029214116664837
iter =  80 , loss =  0.275117563590574 , RMSE train =  1.0292142814548007
iter =  90 , loss =  0.2631475676579398 , RMSE train =  1.0292143196799473
iter =  100 , loss =  0.25765827249171996 , RMSE train =  1.0292143284281277

User-based MF, RMSE =  1.0603799092642756


In [9]:
# Nếu chuẩn hóa dựa trên item:
rs = MF(rate_train, K = 10, lam = 0.1, print_every = 10, learning_rate = 0.75, max_iter = 100, user_based = 0)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print("\nItem-based MF, RMSE = ", RMSE)

iter =  10 , loss =  5.602175194984703 , RMSE train =  1.1757328450391253
iter =  20 , loss =  2.6107111840466026 , RMSE train =  1.0058381384066855
iter =  30 , loss =  1.3211340651786458 , RMSE train =  0.9967209697292754
iter =  40 , loss =  0.7334628174829838 , RMSE train =  0.9962267326775326
iter =  50 , loss =  0.46414182207763477 , RMSE train =  0.9961872256522686
iter =  60 , loss =  0.340645029897718 , RMSE train =  0.9961820194127357
iter =  70 , loss =  0.28401214041482 , RMSE train =  0.9961811148485811
iter =  80 , loss =  0.2580413317079852 , RMSE train =  0.9961809442068909
iter =  90 , loss =  0.24613157253140955 , RMSE train =  0.9961809115366251
iter =  100 , loss =  0.24066996132846208 , RMSE train =  0.9961809053023367

Item-based MF, RMSE =  1.0498047496748308


In [10]:
# Thí nghiệm khi không sử dụng regularization, tức lam = 0:
rs = MF(rate_train, K = 2, lam = 0, print_every = 10, learning_rate = 1, max_iter = 100, user_based = 0)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print("\nItem-based MF, RMSE = ", RMSE)

iter =  10 , loss =  1.167116289379377 , RMSE train =  1.4813935674407408
iter =  20 , loss =  1.0993742886414724 , RMSE train =  1.4610897245970773
iter =  30 , loss =  1.039320677921228 , RMSE train =  1.4419050125158512
iter =  40 , loss =  0.9858104819492165 , RMSE train =  1.4238055684417656
iter =  50 , loss =  0.937905980586558 , RMSE train =  1.4067326848424655
iter =  60 , loss =  0.8948330267203444 , RMSE train =  1.390621638604563
iter =  70 , loss =  0.8559477858019346 , RMSE train =  1.375411184357302
iter =  80 , loss =  0.8207111391094593 , RMSE train =  1.3610227787327172
iter =  90 , loss =  0.7886687899919884 , RMSE train =  1.3473583310707915
iter =  100 , loss =  0.7594356608542788 , RMSE train =  1.3344391712298895

Item-based MF, RMSE =  1.4043132980294388


### Áp dụng lên MovieLens 1M

#### Load data:

In [11]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('./data/ml-1m/ratings.dat', sep='::', names=r_cols, encoding='latin-1')
ratings = ratings_base.as_matrix()

# indices in python start from 0
ratings[:, :2] -= 1

#### Chia data thành 2 tập training và test, sử dụng 1/3 dữ liệu cho test

In [13]:
from sklearn.model_selection import train_test_split

rate_train, rate_test = train_test_split(ratings, test_size=0.33, random_state=42)
print(rate_train.shape, rate_test.shape)

(670140, 4) (330069, 4)


#### Áp dụng Matrix Factorization:

In [14]:
rs = MF(rate_train, K=2, lam=0.1, print_every = 2, learning_rate = 2, max_iter = 10, user_based = 0)
rs.fit()
# evaluate on test data
RMSE = rs.evaluate_RMSE(rate_test)
print("\nItem-based MF, RMSE = ", RMSE)

iter =  2 , loss =  6.7401574089219976 , RMSE train =  1.1137394640042062
iter =  4 , loss =  4.314541445451102 , RMSE train =  1.0004659161985259
iter =  6 , loss =  2.82746439344007 , RMSE train =  0.9778409637708686
iter =  8 , loss =  1.886708803952231 , RMSE train =  0.9740022808241607
iter =  10 , loss =  1.2864786811133475 , RMSE train =  0.9733753862077495

Item-based MF, RMSE =  0.9816181450446788
